In [ ]:
import cv2
import numpy as np
import os
from imutils.contours import sort_contours
import argparse
import imutils
from operator import itemgetter
coords,lines = [],[]
i=0
def capture_handwriting_coordinates():
    cap = cv2.VideoCapture(0) # open the webcam
     # set video height
    while True:
        ret, frame = cap.read() # read the webcam frame
        cv2.imshow("Webcam", frame) # display the frame
        
        # get the mouse events
        events = cv2.setMouseCallback("Webcam", get_coordinates,frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
    cap.release() # release the webcam
    cv2.destroyAllWindows()

def get_coordinates(event, x, y, flags, param):
    global coords,i,lines
    if event == cv2.EVENT_LBUTTONDOWN:
        coords.clear()
        lines.clear()
    elif event==cv2.EVENT_MOUSEMOVE:
        if flags == cv2.EVENT_FLAG_LBUTTON:
            if not coords:
                coords.append((round(x), round(y)))
            else:
                coords.append((round(x), round(y)))
                lines.append(cv2.polylines(param, [np.array(coords)], False, (0,255,0), thickness=5))
                cv2.imshow("Webcam",param)
                cv2.waitKey(1000)
    elif event == cv2.EVENT_LBUTTONUP:
            img = np.zeros((500,500,3), np.uint8)
            lines.append(cv2.polylines(img, [np.array(coords)], False, (255,255,255), thickness=5))
            if cv2.imwrite("C:/Users/ctgfa/CamImage/handwritten("+str(i)+")"+".png", img):
                print("Image Saved!")
def get_image():
    folder_path = 'C:/Users/ctgfa/CamImage/'
    files = os.listdir(folder_path)
    if len(files)>0:
        # Get the creation time of each file
            files_ctime = [(f, os.path.getctime(os.path.join(folder_path, f))) for f in files]

            # Sort the files by their creation time
            files_ctime.sort(key=itemgetter(1))

            # Get the last file
            last_file = files_ctime[-1][0]
            try:
                for file in files:
                    if file != last_file:
                        os.remove(os.path.join(folder_path, file))
            except Exception as e:
                        print(f'Error occured while deleting {file_path} : {e}')
    else: 
            print("There are no files")

capture_handwriting_coordinates()
get_image()

In [19]:
from imutils.contours import sort_contours
import argparse
import imutils

# Load the webcam picture
img = cv2.imread("C:/Users/ctgfa/CamImage/handwritten(0).png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply thresholding
_, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

# Apply Canny edge detection
edges = cv2.Canny(thresh, 100, 150)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Extract the ROI containing the handwriting
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    if w*h > 100:
        roi = img[y:y+h, x:x+w]
        resized_image = cv2.resize(roi, dsize=(28, 28))
        gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
if cv2.imwrite('C:/Users/ctgfa/CamImage/handwritten_image1.jpg', gray_image):
    print("Image Saved")
gray_image = gray_image/255
window_name = 'image'
  
# Using cv2.imshow() method
# Displaying the image
cv2.imshow(window_name, gray_image)
# Save the extracted image
cv2.waitKey(0)
  
# closing all open windows
cv2.destroyAllWindows()

Image Saved


In [5]:

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.optimizers import Adam
from mnist.loader import MNIST

emnist_data = MNIST(path='gzip\\', return_type='numpy')
emnist_data.select_emnist('letters')
x_train, y_train = emnist_data.load_training()
x_test,y_test = emnist_data.load_testing()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = to_categorical(y_train, dtype = "uint8")
y_test = to_categorical(y_test, dtype = "uint8")

# build the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(27, activation='softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# train the model
model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(x_test, y_test))

# evaluate the model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save('alphabet_recognition_model.h5')


Epoch 1/10
975/975 [==============================] - 37s 23ms/step - loss: 1.0092 - accuracy: 0.6958 - val_loss: 0.4094 - val_accuracy: 0.8725
Epoch 2/10
975/975 [==============================] - 22s 22ms/step - loss: 0.5659 - accuracy: 0.8236 - val_loss: 0.3358 - val_accuracy: 0.8974
Epoch 3/10
975/975 [==============================] - 22s 23ms/step - loss: 0.4776 - accuracy: 0.8487 - val_loss: 0.3017 - val_accuracy: 0.9054
Epoch 4/10
975/975 [==============================] - 22s 22ms/step - loss: 0.4379 - accuracy: 0.8613 - val_loss: 0.2889 - val_accuracy: 0.9087
Epoch 5/10
975/975 [==============================] - 21s 22ms/step - loss: 0.4061 - accuracy: 0.8696 - val_loss: 0.2721 - val_accuracy: 0.9147
Epoch 6/10
975/975 [==============================] - 21s 21ms/step - loss: 0.3842 - accuracy: 0.8773 - val_loss: 0.2648 - val_accuracy: 0.9168
Epoch 7/10
975/975 [==============================] - 21s 21ms/step - loss: 0.3629 - accuracy: 0.8833 - val_loss: 0.2618 - val_accuracy:

In [23]:
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
ALPHABET = [chr(i) for i in range(ord('A'), ord('Z') + 1)]
ALPHABET = ALPHABET
print(ALPHABET)
model = keras.models.load_model('handwriting_recognition.h5')
img = cv2.imread("C:/Users/ctgfa/CamImage/handwritten_image1.jpg")
img_arr = tf.keras.preprocessing.image.img_to_array(img)
print(img_arr)
img_arr= img_arr.reshape(-1, 28, 28, 1)
predictions = model.predict(img_arr)
alphabet_index = tf.argmax(predictions[0])
alphabet = ALPHABET[alphabet_index]
print(alphabet)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
[[[  0.   0.   0.]
  [  1.   1.   1.]
  [  0.   0.   0.]
  ...
  [  0.   0.   0.]
  [  0.   0.   0.]
  [  1.   1.   1.]]

 [[  0.   0.   0.]
  [  0.   0.   0.]
  [  3.   3.   3.]
  ...
  [  1.   1.   1.]
  [  1.   1.   1.]
  [255. 255. 255.]]

 [[  2.   2.   2.]
  [  0.   0.   0.]
  [  0.   0.   0.]
  ...
  [  0.   0.   0.]
  [  0.   0.   0.]
  [255. 255. 255.]]

 ...

 [[  0.   0.   0.]
  [  0.   0.   0.]
  [  2.   2.   2.]
  ...
  [  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]]

 [[  3.   3.   3.]
  [  0.   0.   0.]
  [  0.   0.   0.]
  ...
  [  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]]

 [[  0.   0.   0.]
  [  2.   2.   2.]
  [  1.   1.   1.]
  ...
  [  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]]]
1/1 [==============================] - 0s 79ms/step
G
